In [1]:
import numpy as np
import math

In [2]:
import matplotlib.pyplot as plt
%matplotlib inline

# Grid

In [3]:
class Rrt:
    def __init__(self, y0, y01):
        yp = lambda a,b: 0.01 * math.sqrt(0.3*(b+1)) * (a*(58.*b-21.-21.*b*b)+100.)
        y1p = lambda a,b: 1. + a * (b - 0.45 * (b+1.)**2)
        dyda = lambda a,b: 0.01 * math.sqrt(0.3*(b+1)) * (58.*b - 21. - 21.*b*b)
        dydb = lambda a,b: 0.01 * math.sqrt(0.3*(b+1)) * (0.5 * (a*(58.*b - 21. - 21.*b*b) + 100.)/(b+1) + a*(58.-42*b))
        dy1da = lambda a,b: b - 0.45 * (b+1.)**2
        dy1db = lambda a,b: a * (1. - 0.9*(b+1.))

        a = 0.0
        b = 10. / 3.*y0*y0 - 1.

        while True:
            c1 = dyda(a,b)
            c2 = dydb(a,b)
            c3 = y0 - yp(a,b)
            c4 = dy1da(a,b)
            c5 = dy1db(a,b)
            c6 = y01 - y1p(a,b)
            d  = c1*c5 - c2*c4
            da = (c3*c5 - c2*c6) / d
            db = (c1*c6 - c3*c4) / d
            a = a + da
            b = b + db
            if abs(da) < 1.e-5 and abs(db) < 1.e-5: 
                break

        self.aset = a
        self.bset = b

    i0 = lambda self, x: x * (1. + self.aset * (1. - x**2) * (self.bset - x**2))
    i1 = lambda self, x: 1. + self.aset * (self.bset - 3.*(1.+self.bset)*x**2 + 5.*x**4)

In [4]:
Xmax,Zmax,epsr,nsym,curv = 5*[float("nan")]
xf,xn,hx,lx,Im = None, None, float('nan'), 0, 1
rn,rt,rt1,rf,yt,yt1,Jm = 6*[None] + [1]
thn,thf,ht,lt,Km = None, None, float('nan'), 0, 1
Re = float('nan')
cf = float('nan')

In [5]:
def init_r():
    global rn, rt, rt1, rf, yt, yt1
    
    hr = 1.0 / Jm
    rrt = Rrt(1.0, epsr)

    ros = np.linspace(0, 1+hr, Jm+2)
    rn = np.array([rrt.i0(ro) for ro in ros])
    rt = curv + rn * (1.0 - curv)
    rt1 = np.array([rrt.i1(ro)*hr for ro in ros])

    ros -= hr/2
    rf = np.array([rrt.i0(ro) for ro in ros])
    yt = curv + rf * (1.0 - curv)
    yt1 = np.array([rrt.i1(ro)*hr for ro in ros])
    
    
    

def init_th():
    global thn, thf, ht, Km, Zmax
    
    Km = 2**lt
    Zmax = (curv * 2.4 + (1.0 - curv) * math.pi) / nsym
    ht = Zmax / Km
    thn = np.linspace(0, Zmax+ht, Km+2)
    thf = thn - 0.5*ht

def init_x():
    global xn, xf, hx, Im
    
    Im = 2**lx
    hx = Xmax / Im
    xn = np.linspace(0, Xmax+hx, Im+2)
    xf = xn - 0.5*hx

def __str__():
    ret = "## sym geometry module, characteristic units is R and Umax=2*Ubulk \n"
    ret += "Xmax * Ymax * Zmax = %f x %f x %f\n" % (Xmax, 1.0, Zmax)
    ret += "Im * Jm * Km = %d x %d x %d\n" % (Im,Jm,Km)
    ret += "epsr=%f, nsym=%f, hx=%f, ht=%f\n" % (epsr, nsym, hx, ht)
    ret += "Re=%f, cf=%f, " % (Re, cf)
    if curv == 0.0:
        ret += "'pipe'"
    elif curv == 1.0:
        ret += "'duct'"
    else:
        ret += "curv=%f" % curv
    return ret

In [6]:
print __str__()

## sym geometry module, characteristic units is R and Umax=2*Ubulk 
Xmax * Ymax * Zmax = nan x 1.000000 x nan
Im * Jm * Km = 1 x 1 x 1
epsr=nan, nsym=nan, hx=nan, ht=nan
Re=nan, cf=nan, curv=nan


# Calc

In [7]:
def mod_bc(vel):
    u,v,w = vel
        
    vel[:,:,:,0] = vel[:,:,:,-2]
    vel[:,:,:,-1] = vel[:,:,:,1]
    
    for i in xrange(0,Im+1):
        for j in xrange(0,Jm+2):
            u.T[i,j,0] = u.T[i,j,1]
            u.T[i,j,Km+1] = u.T[i,j,Km]
            
        for k in xrange(0,Km+2):
            u.T[i,0,k] = u.T[i,1,k]
            u.T[i,Jm+1,k] = - u.T[i,Jm,k]
            
    for k in xrange(0,Km+2):
        for i in xrange(0,Im+2):
            v.T[i,0,k] = 0.0
            v.T[i,Jm,k] = 0.0
            
    for j in xrange(0,Jm+1):
        for i in xrange(0,Im+2):
            v.T[i,j,0] = v.T[i,j,1]
            v.T[i,j,Km+1] = v.T[i,j,Km]
            
    for j in range(0,Jm+2):
        for i in range(0,Im+2):
            w.T[i,j,0] = 0.0
            w.T[i,j,Km] = 0.0
            
    for k in range(0,Km+1):
        for i in range(0,Im+2):
            w.T[i,0,k] = w.T[i,1,k] * yt[1] / yt[0]
            w.T[i,Jm+1,k] = - w.T[i,Jm,k] * yt[Jm] / yt[Jm+1]

In [8]:
def get_om(vel):
    mod_bc(vel)
    u,v,w = vel
    
    om = np.zeros((3,Km+2,Jm+2,Im+2))
    ox,on,ot = om
    
    for i in range(1,Im+1):
        for j in range(0,Jm+1):
            for k in range(0,Km+1):
                w0 = w.T[i,j,k]
                w1 = w.T[i,j+1,k]
                v0 = v.T[i,j,k]
                v1 = v.T[i,j,k+1]
                ox.T[i,j,k] = ((yt[j+1]*w1 - yt[j]*w0) / rt1[j] - (v1 - v0) / ht) / rt[j]
                
    for k in range(0,Km+1):
        for j in range(1,Jm+1):
            for i in range(0,Im+1):
                u0 = u.T[i,j,k]
                u1 = u.T[i,j,k+1]
                w0 = w.T[i,j,k]
                w1 = w.T[i+1,j,k]
                on.T[i,j,k] = (u1 - u0) / (yt[j] * ht) - (w1 - w0) / hx
                
    for k in range(1,Km+1):
        for j in range(0,Jm+1):
            for i in range(0,Im+1):
                u0 = u.T[i,j,k]
                u1 = u.T[i,j+1,k]
                v0 = v.T[i,j,k]
                v1 = v.T[i+1,j,k]
                ot.T[i,j,k] = (v1 - v0) / hx - (u1 - u0) / rt1[j]
    
    return om

# Read/Write

In [9]:
import pipe_pytools.tools as tools

In [10]:
def init():
    init_x()
    init_r()
    init_th()

In [11]:
def read_scp(fname, is_cf_in=False):
    global cf,Re,Xmax,epsr,lx,Jm,lt,nsym,curv
    t,dt,Dp,Re,Xmax,epsr,lx,Jm,lt,nsym,vel = tools.get_scp(fname)
    curv = 0.0
    if is_cf_in: 
        cf = Dp
    init()
    mod_bc(vel)
    return t,dt,vel

In [12]:
def read_dcp(fname, curv1, is_cf_in=False):
    global cf,Re,Xmax,epsr,lx,Jm,lt,nsym,curv
    t,dt,Dp,Re,Xmax,epsr,lx,Jm,lt,nsym,vel = tools.get_dcp(fname)
    curv = curv1
    if is_cf_in: 
        cf = Dp
    init()
    mod_bc(vel)
    return t,dt,vel

In [13]:
def read_ccp(fname, is_cf_in=False):
    global cf,Re,Xmax,epsr,lx,Jm,lt,nsym,curv
    t,dt,Dp,Re,Xmax,epsr,lx,Jm,lt,nsym,curv,vel = tools.get_ccp(fname)
    if is_cf_in:
        cf = Dp
    init()
    mod_bc(vel)
    return t,dt,vel

In [14]:
def read(fname, curv=-1, is_cf_in=False):
    if fname[-4:] == ".scp":
        return read_scp(fname, is_cf_in)
    if fname[-4:] == ".dcp":
        if curv < 0.0: raise Exception("curv?")
        return read_dcp(fname, curv, is_cf_in)
    if fname[-4:] == ".ccp":
        return read_ccp(fname, is_cf_in)

In [15]:
def write_scp(fname, t, dt, vel):
    if round(nsym) != nsym:
        print "WARNING! nsym is not int"
    if curv != 0.0:
        print "WARNING! geometry is not pipe"
    tools.put_scp(fname, t, dt, cf, Re, Xmax, epsr, lx, Jm, lt, round(nsym), vel)

In [16]:
def write_dcp(fname, t, dt, vel):
    if curv != 0.0 and curv != 1.0:
        print "WARNING! curv is sufficient"
    tools.put_dcp(fname, t, dt, cf, Re, Xmax, epsr, lx, Jm, lt, nsym, vel)

In [17]:
def write_ccp(fname, t, dt, vel):
    tools.put_dcp(fname, t, dt, cf, Re, Xmax, epsr, lx, Jm, lt, nsym, curv, vel)

# Calc

# Elementary

In [18]:
def thmean(vel):
    Vel = np.empty_like(vel)
    Vel[:] = vel[1:-1].mean(0)
    return Vel

In [19]:
def xmean(vel):
    Vel = np.empty_like(vel)
    Vel.T[:] = vel.T[1:-1].mean(0)
    return Vel

In [20]:
def cs_mean(u):
    if len(u.shape) != 3: raise Exception("u.shape should be (Km+2,Jm+2) or (Km+2,Jm+2,..)!")
    return ((u.T[:,:,1:-1].mean(-1) * rf * rf1)[:,1:-1].sum(-1)) / (rf * rf1)[1:-1].sum()

# Plots

In [21]:
import matplotlib.tri as tri

In [22]:
import pipe_pytools.polar_plot as pp

In [19]:
def pipe_contourf(th, r, u, levels, cmap="jet"):
    R = curv / (1.0 - curv)
    radius = R + r
    theta = (th - th.mean()) / (R + 1) - math.pi / 2
    angles = np.repeat(theta[ ..., np.newaxis ], radius.shape[0], axis = 1)

    x = (radius * np.cos( angles )).flatten()
    y = (radius * np.sin( angles )).flatten()
    triang = tri.Triangulation(x, y)

    z = u.flatten()

    scmap = plt.get_cmap(cmap)

    
    plt.tricontourf( triang, z, levels, cmap=scmap)

    ax = plt.gca()
    #ax.xaxis.set_major_locator(plt.NullLocator())
    #ax.yaxis.set_major_locator(plt.NullLocator())
    ax.set_axis_off()
    ax.set_aspect('equal')
    return


In [ ]:
def curv_contourf()

In [39]:
def curv_cs_plot(u):
    

IndentationError: expected an indented block (<ipython-input-39-7b1469a7de1a>, line 2)

In [25]:
def aplot(n):
    a = np.loadtxt("a0.dat", usecols=[0,n], unpack=True)
    plt.plot(a[0],a[n])

In [ ]:
def tw_map(vel):
    VEL = xmean(vel)
    OX = get_om(VEL)
    puls = (vel**2 - VEL**2).sum(0).T[1:-1].mean(0).T**0.5
    duct_cs_ucontourf(puls)

# Test space

In [22]:
t,dt,vel = read("../../experiments/orig-tw/tw2200new/tw2200Xmax5new.scp", is_cf_in=True)

In [23]:
print __str__()

## sym geometry module, characteristic units is R and Umax=2*Ubulk 
Xmax * Ymax * Zmax = 5.000000 x 1.000000 x 1.570796
Im * Jm * Km = 64 x 40 x 32
epsr=0.250000, nsym=2.000000, hx=0.078125, ht=0.049087
Re=2257.670822, cf=0.650426, 'pipe'


In [24]:
curv_cs_ucontourf(vel[0,:,:,10])

TypeError: curv_cs_ucontourf() takes at least 4 arguments (1 given)